In [1]:
import numpy as np
import torch
import experiment
from model import *
from train import *

In [ ]:
source_lr = 0.1
target_lr = 0.01
source_max_iter = 200
target_max_iter = 200

num_experiments = 100

configs = {
    "d": 40,
    "k": 5,
    "n1": 50000,
    "n2": 5000,
    "nT": 100,
    "rho": 1.0,
    "sigma": 0.1,
    "delta": 0.01,
    "c": 1.0,
    "A1": True,
    "A2": True,
    "A3": True,
    "A4": True,
}

In [ ]:
exp = experiment.Experiment(configs)
exp.instantiate(num_experiments)

Parameter assumptions checked
Experiment: A4.1 True, A4.2 True, A4.3 True, A4.4 True


In [6]:
def experimentation(experiment_configs,
        num_experiments=100, 
        source_lr = 0.1,
        target_lr = 0.01,
        source_max_iter = 200,
        target_max_iter = 200
        ):

    exp = experiment.Experiment(experiment_configs)
    exp.instantiate(num_experiments)

    source_X = torch.stack([t.X for t in exp.source_tasks])
    source_y = torch.cat([t.y for t in exp.source_tasks])

    experiment_configs['data'] = exp
    experiment_configs["num_experiments"] = num_experiments
    experiment_configs["source_lr"] = source_lr
    experiment_configs["target_lr"] = target_lr
    experiment_configs["source_max_iter"] = source_max_iter
    experiment_configs["target_max_iter"] = target_max_iter

    optimal_B = get_optimal_B(configs, source_X, source_y)
    experiment_configs['B_hat'] = optimal_B

    loss_list = []

    print("Training target task")
    for task in exp.target_tasks:
        target_model = TaskPredict(configs)
        target_optim = torch.optim.Adam(target_model.parameters(), lr=target_lr)
        target_criterion = torch.nn.MSELoss()
        target_X = task.X
        target_y = task.y

        print(f"Task ID: {task.task_id}")
        train_target = TrainTarget(
            target_model, 
            optimal_B,
            target_optim, 
            target_criterion, 
            target_X, 
            target_y, 
            configs)
        train_target.train(target_max_iter)
        loss_list.append(train_target.final_loss)
    experiment_configs['loss'] = loss_list
    experiment_configs['mean_loss'] = np.mean(loss_list)
    return experiment_configs

In [7]:
result = experimentation(configs)

Parameter assumptions checked
Experiment: A4.1 True, A4.2 True, A4.3 True, A4.4 True
Training source tasks
	 Epoch 0 loss:  21.624
	 Epoch 50 loss:  0.459
	 Epoch 100 loss:  0.049
	 Epoch 150 loss:  0.013
	 Epoch 200 loss:  0.010
Training target task
Task ID: source_0
	 Epoch 0 loss:  18.226
	 Epoch 50 loss:  0.884
	 Epoch 100 loss:  0.185
	 Epoch 150 loss:  0.034
	 Epoch 200 loss:  0.013
Task ID: source_1
	 Epoch 0 loss:  5.653
	 Epoch 50 loss:  0.024
	 Epoch 100 loss:  0.010
	 Epoch 150 loss:  0.010
	 Epoch 185 loss:  0.010
Task ID: source_2
	 Epoch 0 loss:  48.540
	 Epoch 50 loss:  3.364
	 Epoch 100 loss:  0.885
	 Epoch 150 loss:  0.236
	 Epoch 200 loss:  0.054
Task ID: source_3
	 Epoch 0 loss:  15.547
	 Epoch 50 loss:  0.203
	 Epoch 100 loss:  0.010
	 Epoch 150 loss:  0.010
	 Epoch 195 loss:  0.010
Task ID: source_4
	 Epoch 0 loss:  12.346
	 Epoch 50 loss:  0.483
	 Epoch 100 loss:  0.049
	 Epoch 150 loss:  0.012
	 Epoch 200 loss:  0.010
Task ID: source_5
	 Epoch 0 loss:  21.518
	 E

In [11]:
loss_list = np.array(loss_list)
print(loss_list.mean())
print(get_bound(P, sigma, delta))

7.302778
8.211762947458189e-05
